# Neural Network Smartphone Activity Detector

In this activity, you will train a neural network to use smartphone data to predict the activity of the user. 

This dataset has already been separated into input features and target activities. Additional information on the dataset can be found here. 

http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions

### Data Pre-Processing

Prepare the data for the neural network. This includes splitting the data into a training and testing dataset, Scaling the data, and encoding the categorical target values

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Read the input features into `X`
X = pd.read_csv(Path("../Resources/features.csv"), header=None)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.043580,-0.005970,-0.035054,-0.995381,-0.988366,-0.937382,-0.995007,-0.988816,-0.953325,-0.794796,...,-0.012236,-0.314848,-0.713308,-0.112754,0.030400,-0.464761,-0.018446,-0.841559,0.179913,-0.051718
1,0.039480,-0.002131,-0.029067,-0.998348,-0.982945,-0.971273,-0.998702,-0.983315,-0.974000,-0.802537,...,0.202804,-0.603199,-0.860677,0.053477,-0.007435,-0.732626,0.703511,-0.845092,0.180261,-0.047436
2,0.039978,-0.005153,-0.022651,-0.995482,-0.977314,-0.984760,-0.996415,-0.975835,-0.985973,-0.798477,...,0.440079,-0.404427,-0.761847,-0.118559,0.177899,0.100699,0.808529,-0.849230,0.180610,-0.042271
3,0.039785,-0.011809,-0.028916,-0.996194,-0.988569,-0.993256,-0.996994,-0.988526,-0.993135,-0.798477,...,0.430891,-0.138373,-0.491604,-0.036788,-0.012892,0.640011,-0.485366,-0.848947,0.181907,-0.040826
4,0.038758,-0.002289,-0.023863,-0.998241,-0.986774,-0.993115,-0.998216,-0.986479,-0.993825,-0.801982,...,0.137735,-0.366214,-0.702490,0.123320,0.122542,0.693578,-0.615971,-0.848164,0.185124,-0.037080


In [3]:
# Read the target values into `y`
y = pd.read_csv(Path("../Resources/target.csv"))
y.head()

,activity
0,standing
1,standing
2,standing
3,standing
4,standing


In [4]:
y.activity.value_counts()

standing              1423
laying                1413
sitting               1293
walking               1226
walking_upstairs      1073
walking_downstairs     987
stand_to_lie            90
sit_to_lie              75
lie_to_sit              60
lie_to_stand            57
stand_to_sit            47
sit_to_stand            23
Name: activity, dtype: int64

In [5]:
# Split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
# Scale the training and testing input features using StandardScaler
X_scaler = StandardScaler()
X_scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Apply One-hot encoding to the target labels
enc = OneHotEncoder()
enc.fit(y_train)

encoded_y_train = enc.transform(y_train).toarray()
encoded_y_test = enc.transform(y_test).toarray()
encoded_y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

# Build a Deep Neural Network

In [8]:
# Create a sequential model
model = Sequential()

In [9]:
# Add the first layer where the input dimensions are the 561 columns of the training data
model.add(Dense(100, activation='relu', input_dim=X_train_scaled.shape[1]))

In [10]:
# The output layer has 12 columns that are one-hot encoded
y_train.activity.value_counts()
number_outputs = 12

In [11]:
# Add output layer using 12 output nodes
model.add(Dense(number_outputs, activation="softmax"))

In [12]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [13]:
# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               56200     
_________________________________________________________________
dense_1 (Dense)              (None, 12)                1212      
Total params: 57,412
Trainable params: 57,412
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Use the training data to fit (train) the model
# @NOTE: Experiment with the number of training epochs to find the minimum iterations required to achieve a good accuracy
model.fit(
    X_train_scaled,
    encoded_y_train,
    epochs=30,
    shuffle=True,
    verbose=2
)

Train on 5825 samples
Epoch 1/30
5825/5825 - 1s - loss: 0.3812 - accuracy: 0.8635
Epoch 2/30
5825/5825 - 0s - loss: 0.1304 - accuracy: 0.9536
Epoch 3/30
5825/5825 - 0s - loss: 0.0853 - accuracy: 0.9725
Epoch 4/30
5825/5825 - 0s - loss: 0.0626 - accuracy: 0.9779
Epoch 5/30
5825/5825 - 0s - loss: 0.0629 - accuracy: 0.9775
Epoch 6/30
5825/5825 - 0s - loss: 0.0442 - accuracy: 0.9864
Epoch 7/30
5825/5825 - 0s - loss: 0.0398 - accuracy: 0.9875
Epoch 8/30
5825/5825 - 0s - loss: 0.0483 - accuracy: 0.9808
Epoch 9/30
5825/5825 - 0s - loss: 0.0356 - accuracy: 0.9880
Epoch 10/30
5825/5825 - 0s - loss: 0.0246 - accuracy: 0.9916
Epoch 11/30
5825/5825 - 0s - loss: 0.0229 - accuracy: 0.9921
Epoch 12/30
5825/5825 - 0s - loss: 0.0291 - accuracy: 0.9918
Epoch 13/30
5825/5825 - 0s - loss: 0.0394 - accuracy: 0.9868
Epoch 14/30
5825/5825 - 0s - loss: 0.0146 - accuracy: 0.9964
Epoch 15/30
5825/5825 - 0s - loss: 0.0136 - accuracy: 0.9954
Epoch 16/30
5825/5825 - 0s - loss: 0.0145 - accuracy: 0.9940
Epoch 17/30

# Evaluate the Model

In [15]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(X_test_scaled, encoded_y_test, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1942/1 - 0s - loss: 0.3037 - accuracy: 0.9737
Normal Neural Network - Loss: 0.08629389884800148, Accuracy: 0.973738431930542


In [16]:
# Make predictions
predicted = model.predict(X_test_scaled)
predicted = enc.inverse_transform(predicted).flatten().tolist()
results = pd.DataFrame({
    "Actual": y_test.activity.values,
    "Predicted": predicted
})
results.head(10)

,Actual,Predicted
0,walking_upstairs,walking_upstairs
1,laying,laying
2,sitting,sitting
3,sitting,sitting
4,walking,walking
5,sitting,sitting
6,lie_to_sit,lie_to_sit
7,walking_downstairs,walking_downstairs
8,laying,laying
9,sitting,sitting


In [17]:
# Print the Classification Report
from sklearn.metrics import classification_report
print(classification_report(results.Actual, results.Predicted))

                    precision    recall  f1-score   support

            laying       1.00      1.00      1.00       355
        lie_to_sit       0.76      0.87      0.81        15
      lie_to_stand       0.80      0.73      0.76        11
        sit_to_lie       0.89      0.74      0.81        23
      sit_to_stand       1.00      0.75      0.86         4
           sitting       0.95      0.97      0.96       337
      stand_to_lie       0.74      0.78      0.76        18
      stand_to_sit       0.93      0.93      0.93        15
          standing       0.97      0.95      0.96       367
           walking       1.00      0.99      0.99       300
walking_downstairs       0.99      0.99      0.99       230
  walking_upstairs       0.99      1.00      0.99       267

          accuracy                           0.97      1942
         macro avg       0.92      0.89      0.90      1942
      weighted avg       0.97      0.97      0.97      1942

